In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train_data.csv')
df

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,1565607600,30.329370,59.940488,63,4,52,6,0,0,0,0
1,1565568000,30.342845,59.869542,44,0,40,0,4,0,0,0
2,1565568000,30.322632,60.039332,30,0,77,7,0,0,0,1
3,1565568000,30.322632,59.938238,13,0,634,17,32,0,0,7
4,1565568000,30.309158,59.932612,8,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8526521,1565978400,30.324878,59.932612,48,0,15,2,0,0,0,1
8526522,1565978400,30.324878,59.932612,20,1,0,0,0,0,0,0
8526523,1565978400,30.324878,59.932612,54,2,75,7,0,0,0,6
8526524,1565971200,30.324878,59.932612,15,0,350,4,25,0,0,0


In [21]:
sorted(df.lon.unique())[0:7]

[-48.41021066120101,
 0.0,
 18.84665466082756,
 20.486080054345685,
 23.69081983044208,
 24.000738603463308,
 29.40635082565253]

In [31]:
sorted(df.lon.unique())

[-48.41021066120101,
 0.0,
 18.84665466082756,
 20.486080054345685,
 23.69081983044208,
 24.000738603463308,
 29.40635082565253,
 30.08907044158337,
 30.091316229793676,
 30.09356201800397,
 30.09580780621427,
 30.098053594424574,
 30.100299382634866,
 30.102545170845165,
 30.104790959055453,
 30.107036747265763,
 30.10928253547606,
 30.11152832368636,
 30.113774111896657,
 30.11601990010696,
 30.120511476527557,
 30.12275726473785,
 30.125003052948152,
 30.127248841158448,
 30.12949462936875,
 30.13174041757905,
 30.13398620578934,
 30.136231993999644,
 30.138477782209943,
 30.140723570420235,
 30.142969358630538,
 30.14521514684084,
 30.14746093505114,
 30.14970672326144,
 30.15195251147173,
 30.15419829968204,
 30.15644408789233,
 30.158689876102628,
 30.16093566431293,
 30.163181452523236,
 30.16542724073353,
 30.16767302894383,
 30.16991881715413,
 30.17216460536443,
 30.174410393574725,
 30.176656181785024,
 30.178901969995323,
 30.181147758205626,
 30.18339354641592,
 30.1856393

In [33]:
sorted(df.lat.unique())

[-1.3900065324650763,
 0.0,
 35.032142207354845,
 41.30376021953485,
 42.289870330403645,
 42.99689995244231,
 43.68939181898115,
 52.11667860590217,
 54.71500653820782,
 54.77463685891882,
 55.70797485949529,
 55.72189070179645,
 55.75855416422385,
 56.9590331018085,
 57.45166050067814,
 59.656929822910136,
 59.71474042375376,
 59.74756864368603,
 59.75888124818248,
 59.80635237279253,
 59.80748181472322,
 59.8187741296651,
 59.81990315074165,
 59.82103213356388,
 59.82328998444886,
 59.82441885251328,
 59.82893394227072,
 59.83006261908929,
 59.83119125766129,
 59.83231985798753,
 59.833448420068855,
 59.8345769439062,
 59.83570542950036,
 59.83683387685223,
 59.83796228596263,
 59.83909065683245,
 59.840218989462535,
 59.84134728385376,
 59.84247554000695,
 59.84360375792299,
 59.84473193760274,
 59.84586007904705,
 59.84698818225677,
 59.84811624723277,
 59.84924427397591,
 59.85037226248704,
 59.85150021276701,
 59.852628124816725,
 59.85375599863699,
 59.85488383422868,
 59.85601

In [34]:
sorted(df.lon.unique())

[-48.41021066120101,
 0.0,
 18.84665466082756,
 20.486080054345685,
 23.69081983044208,
 24.000738603463308,
 29.40635082565253,
 30.08907044158337,
 30.091316229793676,
 30.09356201800397,
 30.09580780621427,
 30.098053594424574,
 30.100299382634866,
 30.102545170845165,
 30.104790959055453,
 30.107036747265763,
 30.10928253547606,
 30.11152832368636,
 30.113774111896657,
 30.11601990010696,
 30.120511476527557,
 30.12275726473785,
 30.125003052948152,
 30.127248841158448,
 30.12949462936875,
 30.13174041757905,
 30.13398620578934,
 30.136231993999644,
 30.138477782209943,
 30.140723570420235,
 30.142969358630538,
 30.14521514684084,
 30.14746093505114,
 30.14970672326144,
 30.15195251147173,
 30.15419829968204,
 30.15644408789233,
 30.158689876102628,
 30.16093566431293,
 30.163181452523236,
 30.16542724073353,
 30.16767302894383,
 30.16991881715413,
 30.17216460536443,
 30.174410393574725,
 30.176656181785024,
 30.178901969995323,
 30.181147758205626,
 30.18339354641592,
 30.1856393

In [29]:
df[df.lon.isin(sorted(df.lon.unique())[0:1])]

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
37344,1550858400,-48.410211,-1.390007,15,2,17,2,0,0,0,0
164393,1568458800,-48.410211,-1.390007,40,3,25,5,0,0,0,0
165586,1568491200,-48.410211,-1.390007,63,3,341,48,4,0,0,5
165647,1568494800,-48.410211,-1.390007,37,0,231,31,5,0,0,1
315415,1568804400,-48.410211,-1.390007,119,2,43,2,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7944788,1569967200,-48.410211,-1.390007,52,2,61,9,0,0,0,0
8038221,1566648000,-48.410211,-1.390007,40,0,40,7,0,0,0,0
8064643,1568923200,-48.410211,-1.390007,113,1,119,6,8,0,0,0
8126180,1570824000,-48.410211,-1.390007,82,2,14,2,0,0,0,3


In [30]:
df[df.lat.isin(sorted(df.lat.unique())[0:1])]

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
37344,1550858400,-48.410211,-1.390007,15,2,17,2,0,0,0,0
164393,1568458800,-48.410211,-1.390007,40,3,25,5,0,0,0,0
165586,1568491200,-48.410211,-1.390007,63,3,341,48,4,0,0,5
165647,1568494800,-48.410211,-1.390007,37,0,231,31,5,0,0,1
315415,1568804400,-48.410211,-1.390007,119,2,43,2,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7944788,1569967200,-48.410211,-1.390007,52,2,61,9,0,0,0,0
8038221,1566648000,-48.410211,-1.390007,40,0,40,7,0,0,0,0
8064643,1568923200,-48.410211,-1.390007,113,1,119,6,8,0,0,0
8126180,1570824000,-48.410211,-1.390007,82,2,14,2,0,0,0,3
